In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2
import os

def execute_query_safely(sql, con):
    cur = con.cursor()
    
    # try to execute the query
    try:
        cur.execute(sql)
    except:
        # if an exception, rollback, rethrow the exception - finally closes the connection
        cur.execute('rollback;')
        raise
    finally:
        cur.close()
    
    return

aline_path = 'C:/Users/hp/Desktop/help-a-doc/aline/'

concepts_path = 'C:/Users/hp/Desktop/help-a-doc/concepts/'

con = psycopg2.connect(dbname="mimic", user="postgres", password="postgres", host="localhost")

query_schema = 'SET SEARCH_PATH TO public,' + 'mimiciii' + ';'

In [2]:
f = os.path.join(concepts_path,'sepsis/angus.sql')
with open(f) as fp:
    query = ''.join(fp.readlines())

print('Generating materialized view using {} ...'.format(f),end=' ')
execute_query_safely(query_schema + query, con)
print('done.')

f = os.path.join(concepts_path,'demographics/HeightWeightQuery.sql')
with open(f) as fp:
    query = ''.join(fp.readlines())

print('Generating materialized view using {} ...'.format(f),end=' ')
execute_query_safely(query_schema + query, con)
print('done.')

f = os.path.join(aline_path,'aline_vaso_flag.sql')
with open(f) as fp:
    query = ''.join(fp.readlines())
    
# Execute the query
print('Generating materialized view using {} ...'.format(f),end=' ')
execute_query_safely(query_schema + query, con)
print('done.')

'''f = os.path.join(concepts_path+'durations','ventilation-durations.sql')
with open(f) as fp:
    query = ''.join(fp.readlines())
    
# Execute the query
print('Generating materialized view using {} ...'.format(f),end=' ')
execute_query_safely(query_schema + query, con)
print('done.')
'''
f = os.path.join(aline_path,'aline_cohort.sql')
with open(f) as fp:
    query = ''.join(fp.readlines())
    
# Execute the query
print('Generating materialized view using {} ...'.format(f),end=' ')
execute_query_safely(query_schema + query, con)
print('done.')

f = 'aline_bmi.sql'

print('Executing {} ...'.format(f), end=' ')
    
with open(os.path.join(aline_path,f)) as fp:
    query = ''.join(fp.readlines())
        
    execute_query_safely(query_schema + query, con)
        
    print('done')

Generating materialized view using C:/Users/hp/Desktop/help-a-doc/concepts/sepsis/angus.sql ... done.
Generating materialized view using C:/Users/hp/Desktop/help-a-doc/concepts/demographics/HeightWeightQuery.sql ... done.
Generating materialized view using C:/Users/hp/Desktop/help-a-doc/aline/aline_vaso_flag.sql ... done.
Generating materialized view using C:/Users/hp/Desktop/help-a-doc/aline/aline_cohort.sql ... done.
Executing aline_bmi.sql ... done


In [3]:
query = query_schema+'''
SELECT icd9_code, short_title, long_title
FROM D_ICD_DIAGNOSES
'''
df1 = pd.read_sql_query(query, con)

query = query_schema+'''
SELECT DISTINCT hadm_id, icd9_code
FROM DIAGNOSES_ICD
ORDER by hadm_id
'''
df2 = pd.read_sql_query(query, con)

df3 = pd.merge(df1, df2, how='inner', on='icd9_code')

query = query_schema+'''
SELECT DISTINCT hadm_id, drug_type, drug, drug_name_poe, drug_name_generic, formulary_drug_cd, prod_strength, dose_val_rx, dose_unit_rx
FROM PRESCRIPTIONS
ORDER by hadm_id
'''
df4 = pd.read_sql_query(query, con)

df5 = pd.merge(df3, df4, how='inner', on='hadm_id')
df5

,icd9_code,short_title,long_title,hadm_id,drug_type,drug,drug_name_poe,drug_name_generic,formulary_drug_cd,prod_strength,dose_val_rx,dose_unit_rx
0,01186,Pulmon TB NEC-oth test,"Other specified pulmonary tuberculosis, tuberc...",188604,ADDITIVE,Hydrocortisone Na Succinate,None,None,HYDRO100I,100mg Vial (Powder),25,mg
1,01186,Pulmon TB NEC-oth test,"Other specified pulmonary tuberculosis, tuberc...",188604,BASE,0.9% Sodium Chloride,None,None,NS100,100mL Bag,100,mL
2,01186,Pulmon TB NEC-oth test,"Other specified pulmonary tuberculosis, tuberc...",188604,BASE,0.9% Sodium Chloride,None,None,NS1000,1000mL Bag,1000,mL
3,01186,Pulmon TB NEC-oth test,"Other specified pulmonary tuberculosis, tuberc...",188604,BASE,0.9% Sodium Chloride,None,None,NS100CG,100 mL Bag,2,BAG
4,01186,Pulmon TB NEC-oth test,"Other specified pulmonary tuberculosis, tuberc...",188604,BASE,0.9% Sodium Chloride,None,None,NS250,250mL Bag,250,mL
5,01186,Pulmon TB NEC-oth test,"Other specified pulmonary tuberculosis, tuberc...",188604,BASE,0.9% Sodium Chloride,None,None,NS500,500mL Bag,500,mL
6,01186,Pulmon TB NEC-oth test,"Other specified pulmonary tuberculosis, tuberc...",188604,BASE,5% Dextrose,None,None,D5W100,100mL Bag,100,mL
7,01186,Pulmon TB NEC-oth test,"Other specified pulmonary tuberculosis, tuberc...",188604,BASE,5% Dextrose,None,None,D5W50,50mL Bag,50,mL
8,01186,Pulmon TB NEC-oth test,"Other specified pulmonary tuberculosis, tuberc...",188604,BASE,D5NS,None,None,D5NS1000,1000mL Bag,1000,mL
9,01186,Pulmon TB NEC-oth test,"Other specified pulmonary tuberculosis, tuberc...",188604,BASE,Iso-Osmotic Dextrose,None,None,VANCOBASE,200ml Bag,200,mL


In [4]:
query = query_schema + '''
SELECT
co.hadm_id,
co.age,
co.gender,
bmi.weight,
bmi.height,
bmi.bmi
FROM aline_cohort co 
inner join aline_bmi bmi
on co.hadm_id=bmi.hadm_id
'''
df6 = pd.read_sql_query(query, con)
df6

,hadm_id,age,gender,weight,height,bmi
0,100016,55.512886,M,49.7,137.16,0.002642
1,100035,36.538478,M,100.0,177.80,0.003163
2,100063,46.046643,M,58.3,165.10,0.002139
3,100116,21.818996,M,60.7,165.10,0.002227
4,100146,300.003196,F,53.6,157.48,0.002161
5,100232,70.889451,F,75.5,NaN,NaN
6,100267,22.874218,F,61.2,152.40,0.002635
7,100302,48.848435,F,64.5,NaN,NaN
8,100322,32.273465,M,90.7,NaN,NaN
9,100324,22.809962,M,75.0,190.50,0.002067


In [5]:
df7 = pd.merge(df5, df6, how='inner', on='hadm_id')
df7 

,icd9_code,short_title,long_title,hadm_id,drug_type,drug,drug_name_poe,drug_name_generic,formulary_drug_cd,prod_strength,dose_val_rx,dose_unit_rx,age,gender,weight,height,bmi
0,01186,Pulmon TB NEC-oth test,"Other specified pulmonary tuberculosis, tuberc...",188604,ADDITIVE,Hydrocortisone Na Succinate,None,None,HYDRO100I,100mg Vial (Powder),25,mg,55.655312,F,66.0,NaN,NaN
1,01186,Pulmon TB NEC-oth test,"Other specified pulmonary tuberculosis, tuberc...",188604,BASE,0.9% Sodium Chloride,None,None,NS100,100mL Bag,100,mL,55.655312,F,66.0,NaN,NaN
2,01186,Pulmon TB NEC-oth test,"Other specified pulmonary tuberculosis, tuberc...",188604,BASE,0.9% Sodium Chloride,None,None,NS1000,1000mL Bag,1000,mL,55.655312,F,66.0,NaN,NaN
3,01186,Pulmon TB NEC-oth test,"Other specified pulmonary tuberculosis, tuberc...",188604,BASE,0.9% Sodium Chloride,None,None,NS100CG,100 mL Bag,2,BAG,55.655312,F,66.0,NaN,NaN
4,01186,Pulmon TB NEC-oth test,"Other specified pulmonary tuberculosis, tuberc...",188604,BASE,0.9% Sodium Chloride,None,None,NS250,250mL Bag,250,mL,55.655312,F,66.0,NaN,NaN
5,01186,Pulmon TB NEC-oth test,"Other specified pulmonary tuberculosis, tuberc...",188604,BASE,0.9% Sodium Chloride,None,None,NS500,500mL Bag,500,mL,55.655312,F,66.0,NaN,NaN
6,01186,Pulmon TB NEC-oth test,"Other specified pulmonary tuberculosis, tuberc...",188604,BASE,5% Dextrose,None,None,D5W100,100mL Bag,100,mL,55.655312,F,66.0,NaN,NaN
7,01186,Pulmon TB NEC-oth test,"Other specified pulmonary tuberculosis, tuberc...",188604,BASE,5% Dextrose,None,None,D5W50,50mL Bag,50,mL,55.655312,F,66.0,NaN,NaN
8,01186,Pulmon TB NEC-oth test,"Other specified pulmonary tuberculosis, tuberc...",188604,BASE,D5NS,None,None,D5NS1000,1000mL Bag,1000,mL,55.655312,F,66.0,NaN,NaN
9,01186,Pulmon TB NEC-oth test,"Other specified pulmonary tuberculosis, tuberc...",188604,BASE,Iso-Osmotic Dextrose,None,None,VANCOBASE,200ml Bag,200,mL,55.655312,F,66.0,NaN,NaN


In [6]:
df8 = pd.read_csv('symptoms_list.csv', low_memory= False)
result = df8.sort_values(['hadm_id'])
result= result.dropna(axis=1, how='all')
df8 = result.drop_duplicates(keep='first')
df8

,hadm_id,symptom1,symptom2,symptom3,symptom4,symptom5,symptom6,symptom7,symptom8,symptom9,...,symptom16,symptom17,symptom18,symptom19,symptom20,symptom21,symptom22,symptom23,symptom24,symptom25
139777,100006.0,cough,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
139778,100006.0,copd,cough,wheezing,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
139776,100006.0,asthma,cough,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
139774,100006.0,anxiety,delirium,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
139768,100006.0,asthma,copd,cough,multiple myeloma,pneumonia,pulmonary embolism,shortness of breath,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
139767,100006.0,multiple myeloma,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
139773,100006.0,anxiety,cough,wheezing,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
139765,100006.0,tachycardia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
139764,100006.0,endocarditis,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
139763,100006.0,multiple myeloma,pulmonary embolism,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
final_dataset = pd.merge(df7, df8, how='inner', on='hadm_id')
final_dataset

,icd9_code,short_title,long_title,hadm_id,drug_type,drug,drug_name_poe,drug_name_generic,formulary_drug_cd,prod_strength,...,symptom16,symptom17,symptom18,symptom19,symptom20,symptom21,symptom22,symptom23,symptom24,symptom25
0,00845,Int inf clstrdium dfcile,Intestinal infection due to Clostridium difficile,187920,ADDITIVE,Bupivacaine 0.5%,None,None,BUPI5I,0.5%; 30mL Vial,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,00845,Int inf clstrdium dfcile,Intestinal infection due to Clostridium difficile,187920,ADDITIVE,Bupivacaine 0.5%,None,None,BUPI5I,0.5%; 30mL Vial,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,00845,Int inf clstrdium dfcile,Intestinal infection due to Clostridium difficile,187920,ADDITIVE,Bupivacaine 0.5%,None,None,BUPI5I,0.5%; 30mL Vial,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,00845,Int inf clstrdium dfcile,Intestinal infection due to Clostridium difficile,187920,ADDITIVE,Bupivacaine 0.5%,None,None,BUPI5I,0.5%; 30mL Vial,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,00845,Int inf clstrdium dfcile,Intestinal infection due to Clostridium difficile,187920,ADDITIVE,Bupivacaine 0.5%,None,None,BUPI5I,0.5%; 30mL Vial,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,00845,Int inf clstrdium dfcile,Intestinal infection due to Clostridium difficile,187920,ADDITIVE,Bupivacaine 0.5%,None,None,BUPI5I,0.5%; 30mL Vial,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,00845,Int inf clstrdium dfcile,Intestinal infection due to Clostridium difficile,187920,ADDITIVE,Bupivacaine 0.5%,None,None,BUPI5I,0.5%; 30mL Vial,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,00845,Int inf clstrdium dfcile,Intestinal infection due to Clostridium difficile,187920,BASE,D5 1/2NS,None,None,D545NS1000,1000mL Bag,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,00845,Int inf clstrdium dfcile,Intestinal infection due to Clostridium difficile,187920,BASE,D5 1/2NS,None,None,D545NS1000,1000mL Bag,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,00845,Int inf clstrdium dfcile,Intestinal infection due to Clostridium difficile,187920,BASE,D5 1/2NS,None,None,D545NS1000,1000mL Bag,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
final_dataset.to_csv('final_dataset.csv', index=False)